In [ ]:
import os
import glob

import cv2

In [ ]:
#!unzip -d orig /home/tgandor/val2017.zip

def set_up(directory='loss'):
    !rm -r {directory}
    !unzip -q -d {directory} /home/tgandor/val2017.zip
    
def jpgs(directory='orig'):
    return sorted(glob.glob(f'{directory}/**/*.jpg'))


In [ ]:
# Gray Images from val2017:
'''
000000007888.jpg JPEG 638x640 638x640+0+0 8-bit Gray 256c 132380B 0.000u 0:00.000
000000024021.jpg JPEG 640x390 640x390+0+0 8-bit Gray 256c 143004B 0.000u 0:00.000
000000061418.jpg JPEG 640x458 640x458+0+0 8-bit Gray 256c 146703B 0.000u 0:00.000
000000130465.jpg JPEG 640x428 640x428+0+0 8-bit Gray 256c 107975B 0.000u 0:00.000
000000141671.jpg JPEG 640x335 640x335+0+0 8-bit Gray 256c 116094B 0.000u 0:00.000
000000205289.jpg JPEG 640x451 640x451+0+0 8-bit Gray 256c 164786B 0.000u 0:00.000
000000209222.jpg JPEG 640x427 640x427+0+0 8-bit Gray 256c 156441B 0.000u 0:00.000
000000274219.jpg JPEG 464x640 464x640+0+0 8-bit Gray 256c 110257B 0.000u 0:00.000
000000353180.jpg JPEG 640x420 640x420+0+0 8-bit Gray 256c 109535B 0.000u 0:00.000
000000431848.jpg JPEG 378x640 378x640+0+0 8-bit Gray 256c 103826B 0.000u 0:00.000
'''

# Head of sRGB images:
'''
000000000139.jpg JPEG 640x426 640x426+0+0 8-bit sRGB 161811B 0.000u 0:00.000
000000000285.jpg JPEG 586x640 586x640+0+0 8-bit sRGB 335861B 0.000u 0:00.000
000000000632.jpg JPEG 640x483 640x483+0+0 8-bit sRGB 155667B 0.000u 0:00.000
000000000724.jpg JPEG 375x500 375x500+0+0 8-bit sRGB 130107B 0.000u 0:00.000
000000000776.jpg JPEG 428x640 428x640+0+0 8-bit sRGB 176410B 0.000u 0:00.000
000000000785.jpg JPEG 640x425 640x425+0+0 8-bit sRGB 133674B 0.000u 0:00.000
000000000802.jpg JPEG 424x640 424x640+0+0 8-bit sRGB 62406B 0.000u 0:00.000
000000000872.jpg JPEG 621x640 621x640+0+0 8-bit sRGB 317669B 0.000u 0:00.000
000000000885.jpg JPEG 640x427 640x427+0+0 8-bit sRGB 111441B 0.000u 0:00.000
000000001000.jpg JPEG 640x480 640x480+0+0 8-bit sRGB 321136B 0.000u 0:00.000
'''

# Interesting cases - orig sRGB, loss Gray:
'''
orig/val2017/000000001425.jpg - for q == 1
'''

None

In [ ]:
from skimage.metrics import peak_signal_noise_ratio, mean_squared_error, structural_similarity

In [ ]:
cv2.imread('orig/val2017/000000007888.jpg', cv2.IMREAD_ANYCOLOR).shape

In [ ]:
cv2.imread('orig/val2017/000000000139.jpg', cv2.IMREAD_ANYCOLOR).shape

In [ ]:
def degrade(q):
    set_up()
    !mogrify -quality {q} loss/val2017/*.jpg
    
def evaluate(orig_path, loss_path):
    orig = cv2.imread(orig_path, cv2.IMREAD_ANYCOLOR)
    loss = cv2.imread(loss_path, cv2.IMREAD_ANYCOLOR)
    #print(orig_path, orig.shape)
    #print(loss_path, loss.shape)
    
    if orig.shape != loss.shape:
        return {
            'image': os.path.basename(orig_path),
            'orig_shape': orig.shape,
            'loss_shape': loss.shape,
        }
    
    return {
        'image': os.path.basename(orig_path),
        'width': orig.shape[1],
        'height': orig.shape[0],
        'channels': 1 if len(orig.shape) < 3 else orig.shape[2],
        'mean_squared_error': mean_squared_error(orig, loss),
        'peak_signal_noise_ratio': peak_signal_noise_ratio(orig, loss),
        'structural_similarity': 
            structural_similarity(orig, loss, multichannel=True) if len(orig.shape) > 2 
            else structural_similarity(orig, loss),
    }
    

In [ ]:
import json
import numpy as np



In [ ]:
results = []

In [ ]:
%%time

for q in range(1, 101):
    degrade(q)
    for orig, loss in zip(jpgs(), jpgs('loss')):
        part = evaluate(orig, loss)
        part['quality'] = q
        print(part)
        results.append(part)

In [ ]:
len(results)